# Theoretical Foundations of Classification Algorithms for Medical Decision Support

## Introduction

Medical decision support systems increasingly rely on **machine learning classification algorithms** to assist clinicians in diagnosis, prognosis, and risk assessment.  
In such systems, models must not only achieve high predictive performance, but also satisfy important theoretical requirements such as **stability**, **generalization ability**, **robustness to class imbalance**, and **interpretability**.

This notebook presents the **theoretical foundations** of three widely used classification algorithms applied in this project:

- **Logistic Regression**  
- **Support Vector Machine (SVM) with Radial Basis Function (RBF) kernel**  
- **Random Forest**

These models were selected because they represent **complementary learning paradigms**:
- probabilistic linear models,
- margin-based kernel methods,
- and ensemble-based decision tree approaches.

For each algorithm, we focus on:
- the **mathematical formulation** of the model,
- the **optimization objective** and learning process,
- the **statistical interpretation** (when applicable),
- and the **theoretical properties** that make the method suitable for clinical decision support.

This theoretical analysis provides the foundation for the experimental evaluation conducted in the accompanying notebooks:
- one dedicated to a **small clinical dataset**,  
- and another to a **large-scale clinical dataset**.

By clearly separating **theory** from **implementation**, this notebook aims to justify the modeling choices and to facilitate a principled comparison of classification algorithms in a medical context.


### **1. Logistic Regression — mathematical background**

Logistic Regression is a **probabilistic linear classifier** designed for binary classification problems.  
Unlike linear regression, it models the **probability** of class membership rather than a continuous output.

Given an input vector $x$, the model computes a linear score:

$$
z = w^\top x + b
$$

which is then transformed using the **sigmoid (logistic) function**:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}.
$$

The resulting hypothesis is:

$$
h_w(x) = \sigma(w^\top x + b),
$$

which can be interpreted as the conditional probability:

$$
h_w(x) = P(y = 1 \mid x; w).
$$

---

#### Decision rule

Predictions are obtained by thresholding the estimated probability:

$$
\hat{y} =
\begin{cases}
1 & \text{if } h_w(x) \ge 0.5, \\
0 & \text{otherwise}.
\end{cases}
$$

Since $\sigma(z) \ge 0.5 \iff z \ge 0$, the decision boundary is given by:

$$
w^\top x + b = 0,
$$

which corresponds to a **linear separating hyperplane** in the feature space.  
Non-linear decision boundaries can be obtained by introducing non-linear feature mappings.

---

#### Loss function and convexity

Because classification targets are binary, squared error loss is inappropriate.  
Logistic Regression instead minimises the **logistic loss**, also known as **binary cross-entropy**.

For a single sample $(x_i, y_i)$, the loss is defined as:

$$
\ell(h_w(x_i), y_i)
=
- y_i \log(h_w(x_i))
- (1 - y_i)\log(1 - h_w(x_i)).
$$

The empirical risk over the dataset is:

$$
J(w) =
-\frac{1}{n}
\sum_{i=1}^{n}
\Big[
y_i \log(h_w(x_i))
+
(1 - y_i)\log(1 - h_w(x_i))
\Big].
$$

This objective function is **convex**, guaranteeing a unique global optimum and stable convergence.

---

#### Statistical interpretation: Maximum Likelihood Estimation

Logistic Regression has a strong statistical foundation.  
The target variable is assumed to follow a **Bernoulli distribution**:

$$
y_i \sim \text{Bernoulli}(p_i),
\quad
p_i = h_w(x_i).
$$

The likelihood function is:

$$
L(w) =
\prod_{i=1}^{n}
p_i^{y_i}
(1 - p_i)^{1 - y_i}.
$$

Taking the logarithm yields the log-likelihood:

$$
\log L(w) =
\sum_{i=1}^{n}
\Big[
y_i \log(p_i)
+
(1 - y_i)\log(1 - p_i)
\Big].
$$

Minimising the binary cross-entropy loss is therefore equivalent to **maximising the log-likelihood**, meaning that Logistic Regression parameters are estimated using **Maximum Likelihood Estimation (MLE)**.

---

#### Optimisation

Model parameters are typically learned using **gradient descent**.  
The gradient of the cost function with respect to parameter $w_j$ is:

$$
\frac{\partial J(w)}{\partial w_j}
=
\frac{1}{n}
\sum_{i=1}^{n}
\big(h_w(x_i) - y_i\big) x_{ij}.
$$

The update rule is:

$$
w_j \leftarrow w_j - \alpha
\sum_{i=1}^{n}
\big(h_w(x_i) - y_i\big) x_{ij},
$$

where $\alpha > 0$ is the learning rate.

Thanks to the sigmoid function, gradients are **smooth and bounded**, which improves numerical stability compared to vanilla linear classifiers.




---


### **2. Support Vector Machine with RBF kernel — mathematical background**

A Support Vector Machine (SVM) is a **maximum-margin classifier** for binary classification, where class labels are defined as

$$
y_i \in \{-1, +1\}.
$$

The objective of SVM is to find a separating hyperplane that maximises the **margin**, defined as the distance between the decision boundary and the closest training samples from each class (the *support vectors*).

In the linear case, the decision boundary is defined as:

$$
w^\top x + b = 0.
$$

---

#### Maximal margin and hard-margin formulation

For linearly separable data, SVM enforces the constraints:

$$
y_i (w^\top x_i + b) \ge 1,
\qquad i = 1, \dots, n.
$$

The margin is given by:

$$
\rho = \frac{2}{\lVert w \rVert}.
$$

Maximising the margin is equivalent to minimising $\lVert w \rVert$, which leads to the **hard-margin SVM** optimisation problem:

$$
\min_{w, b} \; \frac{1}{2} \lVert w \rVert^2
\quad \text{subject to} \quad
y_i (w^\top x_i + b) \ge 1.
$$

---

#### Soft-margin SVM

In practice, real-world data (including medical data) are rarely perfectly separable.  
To allow violations of the margin constraints, SVM introduces **slack variables** $\xi_i \ge 0$, resulting in the **soft-margin SVM** formulation:

$$
\min_{w, b, \{\xi_i\}}
\; \frac{1}{2} \lVert w \rVert^2
\;+\;
C \sum_{i=1}^{n} \xi_i
$$

subject to:

$$
y_i (w^\top x_i + b) \ge 1 - \xi_i,
\qquad
\xi_i \ge 0,
\qquad
i = 1, \dots, n.
$$

Here:

- $\xi_i$ quantify margin violations and misclassifications,
- $C > 0$ is a regularisation parameter controlling the trade-off between **margin maximisation** and **classification error**:
  - large $C$: low tolerance to errors (narrower margin, lower bias, higher variance),
  - small $C$: higher tolerance to errors (wider margin, higher bias, lower variance).

This formulation corresponds to minimising a regularised **hinge loss** objective.

---

#### Kernel trick and non-linear decision boundaries

Linear SVMs cannot separate data with non-linear structure.  
To address this, SVMs use the **kernel trick**, which implicitly maps inputs into a higher-dimensional feature space:

$$
\phi : \mathbb{R}^d \longrightarrow \mathcal{H}.
$$

Instead of computing $\phi(x)$ explicitly, SVMs rely only on inner products in the feature space, defined via a kernel function:

$$
K(x_i, x_j) = \phi(x_i)^\top \phi(x_j).
$$

This allows SVMs to learn non-linear decision boundaries while keeping the optimisation problem computationally tractable.

---

#### Radial Basis Function (RBF) kernel

In this project, we use the **Radial Basis Function (RBF) kernel**, defined as:

$$
K(x_i, x_j) = \exp\!\left(-\gamma \lVert x_i - x_j \rVert^2\right).
$$

The RBF kernel corresponds to an **infinite-dimensional feature space** and produces smooth, flexible decision boundaries.

The parameter $\gamma > 0$ controls the locality of the kernel:

- large $\gamma$: very local influence of each support vector, leading to highly flexible boundaries and potential overfitting,
- small $\gamma$: smoother, more global boundaries with stronger regularisation.

---

#### Dual formulation and support vectors

Using kernels, SVM optimisation is performed in the **dual formulation**, and the decision function becomes:

$$
\hat{y} =
\operatorname{sign}
\left(
b + \sum_{i \in \mathcal{S}} \alpha_i y_i K(x, x_i)
\right),
$$

where only training samples with non-zero $\alpha_i$ contribute to the decision.  
These samples are called **support vectors**.

---

#### Practical implementation

In `scikit-learn`:

- `C` controls the soft-margin regularisation,
- `gamma` defines the RBF kernel width,
- `probability=True` enables probability estimation via **Platt scaling**.

Hyperparameters $(C, \gamma)$ are selected using `GridSearchCV`, which evaluates candidate models and retains the combination that maximises the **cross-validated ROC-AUC**, a metric particularly suited to imbalanced medical datasets.


### **3. Random Forest — mathematical background**


## 1 Training data (Bootstrap sampling)

Given a dataset:

$$
\mathcal{D} = \{(x_i, y_i)\}_{i=1}^{n}
$$

For each tree \( t = 1, \dots, T \), sample **with replacement**:

$$
\mathcal{D}_t \sim \text{Bootstrap}(\mathcal{D})
$$

 Each tree is trained on a **different dataset**

---

## 2️ Decision tree split (feature randomness)

At each node:

- Randomly select a subset of features:

$$
\mathcal{F}_t \subset \{1,\dots,p\}, \quad |\mathcal{F}_t| = m \ll p
$$

- Choose the best split by minimizing impurity:

$$
\arg\min_{\text{split}} \; \text{Impurity}
$$

### Common impurity measures

#### Gini impurity

$$
G = 1 - \sum_{k=1}^{K} p_k^2
$$

#### Entropy

$$
H = - \sum_{k=1}^{K} p_k \log p_k
$$

---

## 3️ Tree prediction

Each tree produces a prediction:

$$
\hat{y}^{(t)}(x)
$$

---

## 4️ Forest prediction (aggregation)

### Classification (majority vote)

$$
\hat{y}(x) =
\arg\max_{k}
\sum_{t=1}^{T}
\mathbf{1}\big(\hat{y}^{(t)}(x) = k\big)
$$

### Regression (average)

$$
\hat{y}(x) =
\frac{1}{T}
\sum_{t=1}^{T}
\hat{y}^{(t)}(x)
$$

---

## 5️ Why Random Forest works (math intuition)

- Each decision tree is a **high-variance estimator**
- Averaging reduces variance:

$$
\mathrm{Var}\!\left(\frac{1}{T}\sum_{t=1}^{T} f_t(x)\right)
\;\downarrow
$$

- Randomness reduces correlation between trees
- Result: **low variance and strong generalization**

---

## 6️ Bias–Variance perspective

- Single decision tree → low bias, high variance  
- Random Forest → slightly higher bias, **much lower variance**

---



Random Forest is an **ensemble learning method** for classification and regression that combines multiple decision trees to improve generalisation and robustness.  
The core idea is to reduce variance by aggregating the predictions of many weakly correlated models.

---

#### Decision trees as base learners

A Random Forest is built upon **decision trees**, which recursively partition the feature space.  
At each internal node, a split is selected by maximising a purity criterion.

For classification, the most common impurity measure is the **Gini impurity**, defined for a node containing samples from $K$ classes as:

$$
\text{Gini} = 1 - \sum_{k=1}^{K} p_k^2,
$$

where $p_k$ is the proportion of samples belonging to class $k$ in the node.

Alternatively, **entropy** may be used:

$$
\text{Entropy} = - \sum_{k=1}^{K} p_k \log(p_k).
$$

A split is chosen to maximise the **information gain**, i.e. the reduction in impurity after splitting.

---

#### Bootstrap aggregation (bagging)

Random Forest applies **bootstrap aggregation (bagging)** to decision trees.

Given a training dataset of size $n$, each tree is trained on a **bootstrap sample** obtained by sampling $n$ observations **with replacement** from the original dataset.

As a result:
- different trees are trained on different subsets of the data,
- approximately one-third of the samples are left out of each bootstrap sample (*out-of-bag samples*).

Bagging reduces variance by averaging multiple high-variance estimators.

---

#### Random feature selection

To further decorrelate the trees, Random Forest introduces **random feature selection**.

At each split, instead of considering all $m$ input features, only a random subset of size $m_{\text{try}}$ is evaluated.

For classification, a common choice is:

$$
m_{\text{try}} = \sqrt{m}.
$$

This randomness forces trees to explore different structures, reducing correlation between trees and improving ensemble performance.

---

#### Ensemble prediction

Let $\{h_1(x), h_2(x), \dots, h_T(x)\}$ denote the predictions of the $T$ individual trees.

For **classification**, the Random Forest prediction is obtained by **majority voting**:

$$
\hat{y} = \operatorname{mode}\{h_t(x)\}_{t=1}^{T}.
$$

For **probabilistic outputs**, class probabilities are estimated by averaging the class probabilities predicted by individual trees.

---

#### Bias–variance trade-off

Decision trees are low-bias but high-variance models.  
Random Forest reduces variance through averaging while maintaining low bias.

Increasing the number of trees $T$:
- does **not** increase overfitting,
- improves stability,
- converges to a limiting generalisation error.

---

#### Out-of-bag (OOB) error estimation

Because each tree is trained on a bootstrap sample, predictions can be evaluated on the samples not used during training.

The **out-of-bag error** provides an unbiased estimate of the generalisation error without requiring a separate validation set.

---

#### Theoretical particularities

Random Forest exhibits several important theoretical properties:

- Strong resistance to overfitting due to ensemble averaging  
- Ability to model complex non-linear decision boundaries  
- Invariance to monotonic feature transformations  
- Robustness to noise and outliers  

Additionally, Random Forest provides **feature importance measures**, typically based on:
- mean decrease in impurity, or
- permutation importance.

---

#### Practical implementation

In `scikit-learn`, the main hyperparameters include:

- `n_estimators`: number of trees in the forest,
- `max_depth`: maximum depth of each tree,
- `min_samples_split`, `min_samples_leaf`: control tree complexity,
- `max_features`: number of features considered at each split.

In this project, hyperparameters are selected using `GridSearchCV`, and model performance is evaluated using **ROC-AUC**, which is well suited to imbalanced medical datasets.
